# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(filepath,'*'))
    #print(file_path_list)

C:\Users\dayeo\Documents\Data Engineering\Udacity ND\1. Data Modeling\L4 NoSQL Data Models\Project 2 Data Modeling with Apache Cassandra


In [ ]:
print(len(file_path_list))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


NameError: name 'file_path_list' is not defined

In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### Creating a Cluster

In [ ]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [ ]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### <span style="color:darkblue">*The order of the columns in each statement is put in the appropriate order based on the primary key.*</span>

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

We need to think about the query before creating the tables since we are working on the NoSQL database. 
The requirements we have to keep in mind are:
1. to get artist, song title and the length of the song
2. sessionId = 338 and itemInSession = 4

This gives us an idea about the query, and the SELECT statement I came up with is:
`SELECT artist, song_title, length FROM table WHERE sessionId = 338 AND itemInSession = 4`

Now, we can create the table with the necessary columns to run the query, which are `artist`, `song_title`, `length`, `session_id` and `item_in_session`. 

Since the primary key should uniquely identify each row and the constraints are based on the columns `session_id` and `item_in_session`, the partition key should be `session_id` with the clustering column `item_in_session`. If only `session_id` is set to be the primary key, the system will return the filtering error.

In [1]:
table = "CREATE TABLE IF NOT EXISTS music_app_history"
table = table + "(session_id int, item_in_session int, artist varchar, song_title varchar, length float, \
                    PRIMARY KEY(session_id, item_in_session))"
try:
    session.execute(table)
except Exception as e:
    print(e)                    

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO music_app_history (session_id, item_in_session, artist, song_title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [ ]:
query = "SELECT artist, song_title, length FROM music_app_history \
            WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.length)

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

The requirements in the second query are:
1. name of the artist, song title sorted by itemInSession and the first and last name of the user
2. userid = 10 and sessionId = 182

From these, we can come up with the SELECT statement:
`SELECT artist, song_title, fname, lname FROM table WHERE user_id = 10 AND session_id = 182`

Then, the columns we need in the table we will create are:
1. `artist`, `song_title`, `fname` and `lname` for the returning value and
2. `user_id`, `session_id` and `item_in_session` for the constraints

Since the song should be sorted by the column `item_in_session`, the clustering column should be `item_in_session` for this table. The primary key should uniquely identify each column, but there are repeated values in the column `user_id`. Therefore, we need to make it a composite key so that two columns together can uniquely identify each row. In this case, it would be (`user_id`, `session_id`) since those two are the constraints we use in the WHERE clause for this query. 

In [ ]:
table = "CREATE TABLE IF NOT EXISTS sorted_user_info"
table = table + "(user_id int, session_id int, item_in_session int, fname varchar, lname varchar, artist varchar, song_title varchar, \
                    PRIMARY KEY((user_id, session_id), item_in_session))"
try:
    session.execute(table)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO sorted_user_info (user_id, session_id, item_in_session, fname, lname, artist, song_title)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

In [ ]:
query = "SELECT artist, song_title, fname, lname FROM sorted_user_info \
            WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.fname, row.lname)

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

The requirement in the third query is:
1. need first and last name of the user who listened to 'All Hands Against His Own'

Then, the SELECT statement should be:
`SELECT fname, lname FROM table WHERE song_title = 'All Hands Against His Own'`

From that, we can now create the table with the columns `song_title`, `user_id`, `fname` and `lname`. Here, the partition key is `song_title` and the clustering column is `user_id`; `song_title` will partition the data with the value 'All Hands Against His Own' and `user_id` will sort the partitioned data.

In [ ]:
table = "CREATE TABLE IF NOT EXISTS song_user"
table = table + "(song_title varchar, user_id int, fname varchar, lname varchar, \
                    PRIMARY KEY(song_title, user_id))"
try:
    session.execute(table)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_user (song_title, user_id, fname, lname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [ ]:
query = "SELECT fname, lname FROM song_user \
            WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.fname, row.lname)

Here, we will drop the tables we created before closing the session and cluster.

In [ ]:
drop = "DROP TABLE music_app_history"
try:
    rows = session.execute(drop)
except Exception as e:
    print(e)
    
drop1 = "DROP TABLE sorted_user_info"
try:
    rows = session.execute(drop1)
except Exception as e:
    print(e)
    
drop2 = "DROP TABLE song_user"
try:
    rows = session.execute(drop2)
except Exception as e:
    print(e)

In [ ]:
session.shutdown()
cluster.shutdown()